In [2]:
import pandas as pd

In [3]:
sub = pd.read_parquet('SRRF.parquet').iloc[:-1,:]

test = pd.read_parquet('test_sequences.parquet')

In [4]:
test['length'] = test.sequence.str.len()

In [5]:
test.length.value_counts()

207    1000000
177     335823
457       6000
307       2000
Name: length, dtype: int64

In [6]:
test = test.query('future==1')

test['id'] = test.apply(lambda r: list(range(r.id_min,r.id_max+1)), axis=1)

t = test[['sequence_id','id']]

explode = t.explode('id')

explode['id'] = explode.id.astype(int)

explode = explode.sort_values('id')

In [7]:
mask = sub.id.isin(explode.id)

In [9]:
sub2 = sub[mask]

In [11]:
sub2.shape

(101148736, 3)

In [12]:
explode.shape

(210356000, 2)

In [14]:
explode.shape

(210356000, 2)

In [ ]:
merge = explode.merge(sub2,'left')

In [ ]:
del sub

In [9]:
agg = merge.groupby('sequence_id').agg({'reactivity_DMS_MaP': list,'reactivity_2A3_MaP':list})

In [10]:
dms = agg.reactivity_DMS_MaP.apply(pd.Series)

In [11]:
import gc
gc.collect()

0

In [13]:
a3 = agg.reactivity_2A3_MaP.apply(pd.Series)

In [6]:
df = pd.read_parquet('train_data.parquet')

In [10]:
df.columns

Index(['sequence_id', 'sequence', 'experiment_type', 'dataset_name', 'reads',
       'signal_to_noise', 'SN_filter', 'reactivity_0001', 'reactivity_0002',
       'reactivity_0003',
       ...
       'reactivity_error_0197', 'reactivity_error_0198',
       'reactivity_error_0199', 'reactivity_error_0200',
       'reactivity_error_0201', 'reactivity_error_0202',
       'reactivity_error_0203', 'reactivity_error_0204',
       'reactivity_error_0205', 'reactivity_error_0206'],
      dtype='object', length=419)

In [14]:
col_name = [f'reactivity_000{i}' for i in range(1,178)]

In [15]:
len(col_name)

177

In [16]:
len(dms.columns)

177

In [17]:
dms.columns = col_name
a3.columns = col_name

In [18]:
dms['experiment_type'] = 'DMS_MaP'
a3['experiment_type'] = '2A3_MaP'

In [19]:
dms['signal_to_noise'] = 2
a3['signal_to_noise'] = 2

In [21]:
dms.to_parquet('dms.parquet')

In [22]:
a3.to_parquet('a3.parquet')

In [23]:
data = pd.concat([dms.reset_index(),a3.reset_index()])

In [24]:
seq = test[['sequence_id','sequence']]

In [25]:
data = data.merge(seq)

In [27]:
data.to_parquet('public.parquet')